# **BCIE Open Data (CKAN) — Extracción y Preparación de Datos Aprobaciones de Préstamos**

**Objetivo.** Construir un pipeline modular para:
 1) Conectarse a la API de CKAN del BCIE,  
 2) Descargar **un** recurso del grupo **Aprobaciones**,
 3) Normalizar/Tipar columnas (fechas, numéricos, categóricas),  
 4) Guardar salidas (CSV/Parquet),  

**Recurso (resource_id):**
- **Aprobaciones (General)**: `9202bb58-8717-4ca1-83a0-b040d5cf5398`

**Notas de autenticación**:
- Si el recurso es público: no necesitas token (`API_TOKEN=None`).
- Si requiere autenticación: define `API_TOKEN` como variable de entorno o en un archivo `.env` (ver celda de Configuración).

**Salida esperada** (carpeta `./results`):
- `aprobaciones_prestamos.csv`
- `aprobaciones_prestamos.parquet`

In [1]:
import os
import sys
import warnings
import unicodedata
from typing import Optional, Dict, Any, List
import pandas as pd
import numpy as np
import requests
import logging
import sys
!pip install -q kaleido
!pip install -q dataframe_image
import dataframe_image as dfi

## PASO 1: CONFIGURACIÓN INICIAL


In [2]:
logging.info("--- Iniciando Paso 1: Configuración ---")

# --- Silenciar warnings (opcional pero útil) ---
warnings.filterwarnings("ignore", category=UserWarning, module=r"ckanapi")

# URL del portal público de datos abiertos del BCIE
CKAN_BASE = "https://datosabiertos.bcie.org"
# No se requiere token para este recurso público
API_TOKEN = None

# Resource ID específico para "Préstamos – Aprobaciones (General)"
# (Tomado de tu notebook)
RID_PRESTAMOS = "ce88a753-57f5-4266-a57e-394600c8435d"

# Directorio de salida para los archivos generados
OUTDIR = "results"
os.makedirs(OUTDIR, exist_ok=True)

logging.info(f"Directorio de salida: {OUTDIR}")
logging.info(f"ID del Recurso: {RID_PRESTAMOS}")
logging.info(f"API Base: {CKAN_BASE}")

## PASO 2: FUNCIÓN DE EXTRACCIÓN DE DATOS (API)

In [3]:
logging.info("\n--- Iniciando Paso 2: Extracción de Datos (API) ---")

base = CKAN_BASE.rstrip("/")
url = f"{base}/api/3/action/datastore_search"

chunk = 50000
params_common = {"resource_id": RID_PRESTAMOS, "limit": chunk}
rows = []
offset = 0
total = None

headers = {"User-Agent": "bcie-python-client/1.0"}
if API_TOKEN:
    headers["Authorization"] = API_TOKEN

logging.info(f"Conectando a API para resource_id: {RID_PRESTAMOS}...")

while True:
    payload = params_common | {"offset": offset}

    try:
        resp = requests.get(url, params=payload, headers=headers, timeout=60)
        resp.raise_for_status()
        data = resp.json()

        if not data.get("success"):
            raise RuntimeError(f"CKAN error: {data}")

        result = data["result"]
        recs = result["records"]
        rows.extend(recs)

        if total is None:
            total = result["total"]
            logging.info(f"Total registros: {total:,}")

        if not recs or offset + chunk >= total:
            break

        offset += chunk
        logging.info(f"Descargados: {len(rows):,}/{total:,}")

    except requests.RequestException as e:
        logging.error(f"ERROR API: {e}", file=sys.stderr)
        break
    except Exception as e:
        logging.error(f"ERROR procesamiento: {e}", file=sys.stderr)
        break

logging.info(f"\nDescarga completa: {len(rows):,} filas")

df = pd.DataFrame(rows)

logging.info("\n" + "="*60)
logging.info("INFO DATASET")
logging.info("="*60)
logging.info(f"Shape: {df.shape}")
logging.info(f"Columnas: {list(df.columns)}")
logging.info(f"Memoria: {df.memory_usage(deep=True).sum()/1024**2:.1f} MB")

logging.info("\n" + "="*60)
logging.info("ÚLTIMOS 10 REGISTROS")
logging.info("="*60)
logging.info(df.tail(10).to_string(index=False))

csv_path = f"{OUTDIR}/{RID_PRESTAMOS}.csv"
df.to_csv(csv_path, index=False, encoding="utf-8")
logging.info(f"\nGuardado en: {csv_path}")
logging.info("\n¡Listo!")

## PASO 3: FUNCIONES DE LIMPIEZA Y TIPADO


In [4]:
logging.info("\n--- Iniciando Paso 3 (parte 1): Limpieza de Columnas ---")

df = df.copy()

new_cols = []
for c in df.columns:
    s = str(c).strip()
    s_norm = unicodedata.normalize("NFKD", s)
    s_ascii = "".join(ch for ch in s_norm if not unicodedata.combining(ch))
    s_lower = s_ascii.lower()
    s_unders = s_lower.replace(" ", "_").replace("-", "_")
    s_clean = "".join(ch for ch in s_unders if ch.isalnum() or ch == '_')
    if s_clean == "id" and "_id" in s_lower:
        s_clean = "_id"
    new_cols.append(s_clean)

df.columns = new_cols

logging.info("Columnas normalizadas:")
logging.info(list(df.columns))

if 'pais' in df.columns:
    logging.info("Columna 'pais' encontrada exitosamente.")
else:
    logging.warning(f"ADVERTENCIA: 'pais' NO encontrada. Columnas: {list(df.columns)}", file=sys.stderr)

logging.info("\n" + "="*60)
logging.info("ÚLTIMOS 10 REGISTROS (columnas limpias)")
logging.info("="*60)
logging.info(df.tail(10).to_string(index=False))

csv_clean = f"{OUTDIR}/{RID_PRESTAMOS}_limpio.csv"
df.to_csv(csv_clean, index=False, encoding="utf-8")
logging.info(f"\nGuardado limpio en: {csv_clean}")
logging.info("\n¡Paso 3 completado!")

In [5]:
logging.info("\n--- Iniciando Paso 3 (parte 2): Coerción de Tipos ---")

df = df.copy()

if "anio_aprobacion" in df.columns:
    df["anio_aprobacion"] = pd.to_numeric(df["anio_aprobacion"], errors="coerce").astype("Int64")

if "monto_bruto_usd" in df.columns:
    df["monto_bruto_usd"] = pd.to_numeric(df["monto_bruto_usd"], errors="coerce").astype(float)

if "cantidad_aprobaciones" in df.columns:
    df["cantidad_aprobaciones"] = pd.to_numeric(df["cantidad_aprobaciones"], errors="coerce").astype("Int64")

logging.info("Tipos aplicados:")
logging.info(df.dtypes)

logging.info("\n" + "="*60)
logging.info("INFO DATASET (tipos corregidos)")
logging.info("="*60)
logging.info(f"Shape: {df.shape}")
logging.info(f"Memoria: {df.memory_usage(deep=True).sum()/1024**2:.1f} MB")

logging.info("\n" + "="*60)
logging.info("ÚLTIMOS 10 REGISTROS")
logging.info("="*60)
logging.info(df.tail(10).to_string(index=False))

csv_final = f"{OUTDIR}/{RID_PRESTAMOS}_final.csv"
df.to_csv(csv_final, index=False, encoding="utf-8")
logging.info(f"\nGuardado final en: {csv_final}")
logging.info("\n¡Paso 3 completado al 100%!")

## PASO 4: FUNCIONES DE CLASIFICACIÓN Y AGREGACIÓN


In [6]:
logging.info("\n--- Iniciando Paso 4 (parte 1): Clasificación de Socios ---")

def _norm_txt(s):
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    s_norm = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s_norm if not unicodedata.combining(ch))

FUNDADORES = {_norm_txt(x) for x in ["Guatemala", "El Salvador", "Honduras", "Nicaragua", "Costa Rica"]}
REG_NO_F   = {_norm_txt(x) for x in ["República Dominicana", "Panamá", "Belice"]}
EXTRAREG   = {_norm_txt(x) for x in ["México", "República de China (Taiwán)", "Argentina", "Colombia", "España", "Cuba", "República de Corea"]}

def clasificar_socio(row):
    pais_norm = _norm_txt(row["pais"])
    sector_norm = _norm_txt(row["sector_institucional"])

    if pais_norm == "regional":
        return "Regionales"
    if pais_norm == "institucional" or sector_norm == "institucional":
        return "Institucionales"
    if pais_norm in FUNDADORES:
        return "Fundadores"
    if pais_norm in REG_NO_F:
        return "Regional No Fundadores"
    if pais_norm in EXTRAREG:
        return "Extraregionales"
    if pd.isna(row["pais"]) or pais_norm in {"", "sin pais"}:
        return "Sin País"
    return "Otros"

df["tipo_socio"] = df.apply(clasificar_socio, axis=1)

df = df[["_id", "anio_aprobacion", "sector_institucional", "pais", "tipo_socio", "monto_bruto_usd", "cantidad_aprobaciones"]]

logging.info("Clasificación 'tipo_socio' aplicada y columnas reordenadas.")
logging.info("\nDistribución de tipo_socio:")
logging.info(df["tipo_socio"].value_counts())

logging.info("\n" + "="*80)
logging.info("ÚLTIMOS 10 REGISTROS (orden solicitado)")
logging.info("="*80)
logging.info(df.tail(10).to_string(index=False))

csv_clasif = f"{OUTDIR}/{RID_PRESTAMOS}_clasificado.csv"
df.to_csv(csv_clasif, index=False, encoding="utf-8")
logging.info(f"\nGuardado con clasificación y orden en: {csv_clasif}")
logging.info("\n¡Paso 4 (parte 1) completado!")

In [7]:
logging.info("\n--- Iniciando Paso 4 (parte 2): Agregación, Promedio y Formato Final ---")

df_agg = (
    df.groupby(["anio_aprobacion", "tipo_socio", "pais", "sector_institucional"], as_index=False)
      .agg(
          monto_total_usd_aprobados=("monto_bruto_usd", "sum"),
          cantidad_total_aprobados=("cantidad_aprobaciones", "sum")
      )
)

df_agg["promedio_aprobacion_usd"] = (
    df_agg["monto_total_usd_aprobados"] / df_agg["cantidad_total_aprobados"]
).where(df_agg["cantidad_total_aprobados"] > 0, np.nan)

# Orden descendente por año (más reciente primero)
df_agg = df_agg.sort_values("anio_aprobacion", ascending=False)

df_agg = df_agg.rename(columns={
    "anio_aprobacion": "Año",
    "tipo_socio": "Tipo de Socio",
    "pais": "País",
    "sector_institucional": "Sector Institucional",
    "monto_total_usd_aprobados": "Monto Total (USD)",
    "cantidad_total_aprobados": "Cantidad Total",
    "promedio_aprobacion_usd": "Promedio por Aprobación (USD)"
})

df_agg = df_agg[[
    "Año", "Sector Institucional", "País", "Tipo de Socio",
    "Monto Total (USD)", "Cantidad Total", "Promedio por Aprobación (USD)"
]]

# Formateo bonito
def fmt(x):
    if pd.isna(x):
        return ""
    return f"{x:,.0f}"

styled = df_agg.head(10).style.format({
    "Monto Total (USD)": fmt,
    "Cantidad Total": "{:.0f}",
    "Promedio por Aprobación (USD)": fmt
}).set_properties(**{
    'text-align': 'center',
    'font-family': 'Arial',
    'font-size': '11pt'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#2c3e50'), ('color', 'white'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('border', '1px solid #ddd')]},
    {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]}
])

logging.info("Agregación completa con promedio calculado.")
logging.info(f"\nTotal de filas agregadas: {len(df_agg):,}")
logging.info(f"Años cubiertos: {df_agg['Año'].max()} - {df_agg['Año'].min()} (orden descendente)")

logging.info("\n" + "="*120)
logging.info("TOP 10 REGISTROS MÁS RECIENTES ")
logging.info("="*120)
display(styled)

csv_final = f"{OUTDIR}/tabla_final.csv"
excel_final = f"{OUTDIR}/tabla_final.xlsx"
parquet_final = f"{OUTDIR}/tabla_final.parquet"

df_agg.to_csv(csv_final, index=False, encoding="utf-8")
df_agg.to_excel(excel_final, index=False, engine="openpyxl")
df_agg.to_parquet(parquet_final, index=False)

logging.info(f"\n¡TODO LISTO! Archivos generados (ordenados por año descendente):")
logging.info(f"   CSV     → {csv_final}")
logging.info(f"   Excel   → {excel_final}")
logging.info(f"   Parquet → {parquet_final}")

logging.info("\n¡Paso 4 completado al 100%! Listo para Power BI, Tableau o Python.")

,Año,Sector Institucional,País,Tipo de Socio,Monto Total (USD),Cantidad Total,Promedio por Aprobación (USD)
608,2025,Sector Público,Regional,Regionales,"37,187,479",1,"37,187,479"
603,2025,Sector Público,Honduras,Fundadores,"465,000,000",2,"232,500,000"
602,2025,Sector Público,Guatemala,Fundadores,"60,000,000",1,"60,000,000"
601,2025,Sector Privado,Guatemala,Fundadores,"15,000,000",1,"15,000,000"
600,2025,Sector Público,El Salvador,Fundadores,"350,000,000",3,"116,666,667"
606,2025,Sector Público,Panamá,Regional No Fundadores,"75,000,000",1,"75,000,000"
605,2025,Sector Privado,Panamá,Regional No Fundadores,"20,000,000",1,"20,000,000"
607,2025,Sector Público,República Dominicana,Regional No Fundadores,"80,900,000",1,"80,900,000"
604,2025,Sector Público,Nicaragua,Fundadores,"235,000,000",1,"235,000,000"
597,2025,Sector Público,Argentina,Extraregionales,"100,000,000",1,"100,000,000"


# Caso 1 (Parte 2): Forecasting con XGBoost y Lag Features

Este notebook es la continuación del "Caso 1: Forecasting de Aprobaciones BCIE".

### Objetivo de este Notebook

Nos enfocaremos en construir, entrenar y validar el modelo **XGBoost** con *lag features*. Este modelo es la segunda mitad de nuestro ensamble híbrido y es clave para capturar las interacciones no lineales y los patrones históricos que Prophet podría pasar por alto.

### Plan de Acción

1.  **Cargar** los datos limpios (`tabla_final.parquet`).
2.  **Feature Engineering:** Crear *lags* (ej. monto año anterior) y *rolling averages*.
3.  **Entrenar** un modelo XGBoost usando un set de validación temporal.
4.  **Evaluar** el modelo y analizar la importancia de las features.

---

**Siguiente celda → Importar librerías y cargar el Parquet (ÓPTIMO para ML)** ¿Listos para ver el futuro del BCIE? ¡Ejecuten! 🚀

In [8]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import sys
from IPython.display import display, HTML
!pip install -q kaleido
!pip install -q dataframe_image
import dataframe_image as dfi

# Modelos y Métricas
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# Variable global del notebook anterior
OUTDIR = "results"

logging.info("Librerías de ML (Pandas, XGBoost, Sklearn) y visualización cargadas.")
logging.info(f"XGBoost version: {xgb.__version__}")

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

logging.info("Sistema de Logging configurado.")

In [9]:
logging.info("\n--- Caso 1: Forecasting BCIE 2026-2030 | Paso 1: Carga del Parquet (ÓPTIMO para ML) ---")

parquet_path = f"{OUTDIR}/tabla_final.parquet"

try:
    # En un notebook nuevo, SIEMPRE cargamos desde el archivo
    logging.info(f"Cargando desde Parquet (rápido y seguro): {parquet_path}")
    df_ml = pd.read_parquet(parquet_path)
except FileNotFoundError:
    logging.error(f"ERROR: No se encontró el archivo {parquet_path}. Asegúrate de ejecutar el notebook de preparación de datos primero.")
    # Detener ejecución o manejar error
    raise

# Forzamos orden descendente por año (más reciente primero)
df_ml = df_ml.sort_values("Año", ascending=False).reset_index(drop=True)

logging.info(f"\n¡Datos cargados perfectamente desde Parquet!")
logging.info(f"Shape: {df_ml.shape}")
# Usar la fecha actual simulada
logging.info(f"Años: {df_ml['Año'].min()} → {df_ml['Año'].max()} (hoy: {pd.Timestamp.now().strftime('%B %Y')})")
logging.info(f"Memoria: {df_ml.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
logging.info(f"Tipos conservados:\n{df_ml.dtypes}")

logging.info("\n" + "="*100)
logging.info("VISTA PREVIA (10 años más recientes)")
logging.info("="*100)
styled_preview = df_ml.head(10).style.format({
    "Monto Total (USD)": lambda x: f"{x:,.0f}",
    "Cantidad Total": "{:.0f}",
    "Promedio por Aprobación (USD)": lambda x: f"{x:,.0f}" if pd.notna(x) else ""
}).set_properties(**{
    'text-align': 'center',
    'font-family': 'Arial',
    'font-size': '11pt'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#1f77b4'), ('color', 'white'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('border', '1px solid #ddd')]},
])
display(styled_preview)

logging.info(f"\n¡Listo! Ahora df_ml está 100% optimizado para Machine Learning.")
logging.info("   → Tipos perfectos")
logging.info("   → Orden cronológico descendente")
logging.info("   → Parquet = velocidad relámpago en próximos pasos")

logging.info("\nSiguiente celda → Paso 2: Feature Engineering temporal (lags, rolling, crisis flags)")
logging.info("¡Ejecuten y veremos el 2026 antes que nadie! 🚀")

,Año,Sector Institucional,País,Tipo de Socio,Monto Total (USD),Cantidad Total,Promedio por Aprobación (USD)
0,2025,Sector Público,Panamá,Regional No Fundadores,"75,000,000",1,"75,000,000"
1,2025,Sector Privado,Panamá,Regional No Fundadores,"20,000,000",1,"20,000,000"
2,2025,Sector Público,República Dominicana,Regional No Fundadores,"80,900,000",1,"80,900,000"
3,2025,Sector Privado,Guatemala,Fundadores,"15,000,000",1,"15,000,000"
4,2025,Sector Público,Guatemala,Fundadores,"60,000,000",1,"60,000,000"
5,2025,Sector Público,Honduras,Fundadores,"465,000,000",2,"232,500,000"
6,2025,Sector Público,Regional,Regionales,"37,187,479",1,"37,187,479"
7,2025,Sector Público,El Salvador,Fundadores,"350,000,000",3,"116,666,667"
8,2025,Sector Público,Argentina,Extraregionales,"100,000,000",1,"100,000,000"
9,2025,Sector Público,Colombia,Extraregionales,"75,000,000",1,"75,000,000"


In [10]:
logging.info("\n--- [TEST] Validando Paso 1 ---")
assert 'df_ml' in locals() and not df_ml.empty, "El DataFrame df_ml está vacío."
assert 'Año' in df_ml.columns, "La columna 'Año' no se encontró."
assert 'Monto Total (USD)' in df_ml.columns, "La columna 'Monto Total (USD)' no se encontró."
assert 'País' in df_ml.columns, "La columna 'País' no se encontró."
logging.info("✅ [Assert OK] Paso 1: Datos cargados y columnas clave validadas.")

## Paso 2: Feature Engineering (Lags, Rolling, Crisis Flags)

Este es el paso más crucial para cualquier modelo de forecasting, especialmente XGBoost. Vamos a crear la "inteligencia" del modelo.

### ¿Por qué este paso es CRUCIAL para forecasting financiero?

-   **Lags** → "Memoria" del modelo (qué pasó el año pasado influye este año).
-   **Rolling** → Tendencias suaves (media 3 años, volatilidad).
-   **Growth rates** → % cambio (crecimiento explosivo o caída).
-   **Crisis flags** → Shocks externos (COVID, huracanes, crisis 2008).

Sin esto, XGBoost no tiene contexto histórico y no puede predecir con precisión.

---

**Siguiente celda → Ejecución del Feature Engineering completo.**

In [11]:
# =============================================
# --- PASO 2: FEATURE ENGINEERING COMPLETO ---
# =============================================
logging.info("Iniciando Paso 2: Feature Engineering (Lags, Rolling, Crisis)...")

# =============================================
# PARTE 1: Carga y orden básico
# =============================================
logging.info("\n--- PARTE 1: Carga y orden ---")
# df_ml viene de la celda anterior
df_fe = df_ml.copy()
# ¡¡CRÍTICO!! Ordenar por AÑO ASCENDENTE para que los lags y rolling funcionen
df_fe = df_fe.sort_values("Año").reset_index(drop=True)
logging.info(f"Shape para FE: {df_fe.shape}")
logging.info(f"Ordenado: {df_fe['Año'].iloc[0]} -> {df_fe['Año'].iloc[-1]}")

In [12]:
# =============================================
# PARTE 2: Grupos y LAGS (para 'Monto', 'Cantidad' y 'Promedio')
# =============================================
logging.info("\n--- PARTE 2: Lags ---")
group_cols = ["País", "Sector Institucional", "Tipo de Socio"]

# Creamos lags de 1, 2 y 3 años
for lag in [1, 2, 3]:
    df_fe[f"monto_lag_{lag}"] = df_fe.groupby(group_cols)["Monto Total (USD)"].shift(lag)
    df_fe[f"cant_lag_{lag}"] = df_fe.groupby(group_cols)["Cantidad Total"].shift(lag)
    df_fe[f"prom_lag_{lag}"] = df_fe.groupby(group_cols)["Promedio por Aprobación (USD)"].shift(lag)

logging.info("Lags [1, 2, 3] creados para Monto, Cantidad y Promedio.")

In [13]:
# =============================================
# PARTE 3: Rolling y Growth (sobre la variable de Monto)
# =============================================
logging.info("\n--- PARTE 3: Rolling y Growth ---")
roll = 3
# .shift(1) es VITAL para evitar data leakage (usamos datos *hasta* el año anterior)
df_fe["monto_roll_mean_3"] = df_fe.groupby(group_cols)["Monto Total (USD)"].transform(
    lambda x: x.rolling(roll, min_periods=1).mean().shift(1)
)
df_fe["monto_roll_std_3"] = df_fe.groupby(group_cols)["Monto Total (USD)"].transform(
    lambda x: x.rolling(roll, min_periods=1).std().shift(1)
)
df_fe["cant_roll_mean_3"] = df_fe.groupby(group_cols)["Cantidad Total"].transform(
    lambda x: x.rolling(roll, min_periods=1).mean().shift(1)
)

# Tasa de crecimiento (usa el lag 1)
df_fe["monto_growth_1y"] = df_fe["Monto Total (USD)"].pct_change()
df_fe["monto_growth_3y"] = df_fe["Monto Total (USD)"].pct_change(periods=3)
# Asegurar que el growth sea por grupo
df_fe["monto_growth_1y_grouped"] = df_fe.groupby(group_cols)["Monto Total (USD)"].pct_change() * 100
df_fe["monto_growth_3y_grouped"] = df_fe.groupby(group_cols)["Monto Total (USD)"].pct_change(periods=3) * 100

logging.info("Rolling means, std, y growth rates (pct_change) OK.")

In [14]:
# =============================================
# PARTE 4: Crisis flags (Variables Exógenas)
# =============================================
logging.info("\n--- PARTE 4: Crisis flags ---")
df_fe["crisis_covid"]       = ((df_fe["Año"] >= 2020) & (df_fe["Año"] <= 2021)).astype(int)
df_fe["crisis_2008"]        = (df_fe["Año"] == 2008).astype(int)
df_fe["crisis_nicaragua"]   = ((df_fe["Año"] >= 2018) & (df_fe["País"] == "Nicaragua")).astype(int)
df_fe["post_covid_boom"]    = (df_fe["Año"] >= 2022).astype(int)

logging.info("Flags de crisis (COVID, 2008, Nic-2018, Post-COVID) creadas.")

In [15]:
# =============================================
# PARTE 5: Fill NA (Rellenar con Mediana de Grupo)
# =============================================
logging.info("\n--- PARTE 5: Fill NA ---")
lag_roll_cols = [c for c in df_fe.columns if any(k in c for k in ["lag", "roll", "growth"])]
logging.info(f"Columnas a rellenar ({len(lag_roll_cols)}): {lag_roll_cols[:3]}...")

logging.info("Rellenando NaNs con la mediana del grupo (estrategia avanzada)...")
for col in lag_roll_cols:
    # Calcula la mediana para cada grupo y la usa para rellenar los NaN de ese mismo grupo
    # MODIFICACIÓN: Si la columna es Int64, redondear la mediana y convertir a Int64
    if str(df_fe[col].dtype) == 'Int64': # Check for nullable integer type
        # Check if median is NA before rounding/casting
        df_fe[col] = df_fe.groupby(group_cols)[col].transform(lambda x:
            x.fillna(x.median().round().astype(int)) if not pd.isna(x.median()) else x)
    else:
        df_fe[col] = df_fe.groupby(group_cols)[col].transform(lambda x: x.fillna(x.median()))

# Si algún grupo entero era NaN (ej. primer año), la mediana será NaN.
# Rellenar esos NaNs restantes con 0 (como fallback final).
df_fe[lag_roll_cols] = df_fe[lag_roll_cols].fillna(0)
logging.info("NA rellenados con la mediana del grupo (y 0 como fallback).")

/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, ou

In [16]:
# =============================================
# PARTE 6: One-hot encoding (para XGBoost)
# =============================================
logging.info("\n--- PARTE 6: One-hot encoding ---")
# Guardamos las columnas de texto originales antes de borrarlas
df_original_text = df_fe[["Año", "País", "Sector Institucional", "Tipo de Socio"]].copy()

cat_cols = ["Sector Institucional", "Tipo de Socio", "País"]
logging.info(f"Creando dummies para: {cat_cols}")
dummies = pd.get_dummies(df_fe[cat_cols], drop_first=True, dtype=int)
# Unimos las dummies y borramos las de texto
df_fe = pd.concat([df_fe.drop(columns=cat_cols), dummies], axis=1)

logging.info(f"Dummies creadas: {dummies.shape[1]}")
logging.info(f"Total columnas finales para ML: {df_fe.shape[1]}")
logging.info(f"Columnas de texto eliminadas del df_fe: {'País' not in df_fe.columns}")

In [17]:
# =============================================
# PARTE 7: Guardar Parquet de FEATURES (para el modelo)
# =============================================
logging.info("\n--- PARTE 7: Guardar Parquet de Features (para ML) ---")
fe_path = f"{OUTDIR}/tabla_final_features.parquet"
df_fe.to_parquet(fe_path, index=False, compression="gzip")
logging.info(f"¡DATOS PARA MODELO GUARDADOS! → {fe_path}")
logging.info("Este archivo contiene SÓLO NÚMEROS (One-Hot) y es el que usará XGBoost.")

In [18]:
# =============================================
# PARTE 8 FINAL: RECONSTRUIR VISTA HUMANA + GUARDAR
# =============================================
logging.info("\n--- PARTE 8: Reconstruir Vista Humana (para Power BI / Tableau) ---")

# Reconstruir la vista "bonita" (humana)
# Tomamos el df_fe (con todas las features) y le pegamos las columnas de texto originales
df_full_vista = df_fe.copy()

# df_original_text fue creado en la PARTE 6
# Alineamos los índices para pegar las columnas de texto de vuelta
df_original_text = df_original_text.reset_index(drop=True)
df_full_vista = df_full_vista.reset_index(drop=True)

df_full_vista = pd.concat([df_original_text, df_full_vista.drop(columns=["Año"])], axis=1)

# Últimos 15 registros (más recientes)
latest = df_full_vista.sort_values("Año", ascending=False).head(15)

# Asegúrate que 'monto_growth_1y_grouped' existe (creada en Parte 3)
if 'monto_growth_1y_grouped' not in latest.columns:
    logging.info("Advertencia: 'monto_growth_1y_grouped' no se encontró, usando 'monto_growth_1y'")
    growth_col = 'monto_growth_1y'
else:
    growth_col = 'monto_growth_1y_grouped'

display_cols = ["Año", "País", "Sector Institucional", "Tipo de Socio",
                "Monto Total (USD)", "monto_lag_1", growth_col]

styled = latest[display_cols].style.format({
    "Monto Total (USD)": lambda x: f"{x:,.0f}",
    "monto_lag_1": lambda x: f"{x:,.0f}",
    growth_col: "{:.1f}%"
}).set_properties(**{
    'text-align': 'center', 'color': 'black', 'background-color': 'white', 'font-size': '11pt'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#1f77b4'), ('color', 'white'), ('font-weight', 'bold')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f2f2f2')]}
]).background_gradient(cmap='Greens', subset=["Monto Total (USD)"])

logging.info("ÚLTIMOS 15 REGISTROS (todos los países - más recientes primero)")
display(styled)

# GUARDAR EN 3 FORMATOS
csv_path = f"{OUTDIR}/vista_todos_paises_features.csv"
excel_path = f"{OUTDIR}/vista_todos_paises_features.xlsx"
parquet_path = f"{OUTDIR}/vista_todos_paises_features.parquet"

# df_full_vista es el DataFrame que tiene las features Y el texto
df_full_vista.to_csv(csv_path, index=False, encoding="utf-8")
df_full_vista.to_excel(excel_path, index=False, engine="openpyxl")
df_full_vista.to_parquet(parquet_path, index=False)

logging.info(f"\n¡GUARDADO PARA TABLEAU / POWER BI!")
logging.info(f"CSV     → {csv_path}")
logging.info(f"EXCEL   → {excel_path}")
logging.info(f"PARQUET → {parquet_path}")

logging.info("\nPASO 2 100% COMPLETADO")
# fe_path fue creado en la PARTE 7
logging.info(f"df_fe = Datos para ML (One-Hot) → {fe_path}")
logging.info(f"df_full_vista = Vista Humana (con texto) → {parquet_path}")
logging.info("\n¡Listo para Paso 3: Entrenamiento de XGBoost!")

,Año,País,Sector Institucional,Tipo de Socio,Monto Total (USD),monto_lag_1,monto_growth_1y_grouped
608,2025,Panamá,Sector Público,Regional No Fundadores,"75,000,000","240,000,000",-68.8%
603,2025,República Dominicana,Sector Público,Regional No Fundadores,"80,900,000","100,000,000",-19.1%
602,2025,Guatemala,Sector Privado,Fundadores,"15,000,000","19,000,000",-21.1%
601,2025,Panamá,Sector Privado,Regional No Fundadores,"20,000,000","85,000,000",-76.5%
600,2025,Guatemala,Sector Público,Fundadores,"60,000,000","170,000,000",-64.7%
606,2025,Regional,Sector Público,Regionales,"37,187,479",0,0.0%
605,2025,El Salvador,Sector Público,Fundadores,"350,000,000","263,900,000",32.6%
607,2025,Honduras,Sector Público,Fundadores,"465,000,000","476,300,000",-2.4%
604,2025,Argentina,Sector Público,Extraregionales,"100,000,000","230,000,000",-56.5%
597,2025,Colombia,Sector Público,Extraregionales,"75,000,000","250,000,000",-70.0%


In [19]:
logging.info("\n--- [TEST] Validando Paso 2 ---")
assert 'df_fe' in locals() and not df_fe.empty, "El DataFrame de features df_fe está vacío."
assert 'monto_lag_1' in df_fe.columns, "Fallo en PARTE 2: No se crearon las columnas de lag."
assert 'monto_roll_mean_3' in df_fe.columns, "Fallo en PARTE 3: No se crearon las columnas rolling."
assert 'crisis_covid' in df_fe.columns, "Fallo en PARTE 4: No se crearon las flags de crisis."

# Definimos las columnas de lag/roll localmente para el test
lag_roll_cols = [c for c in df_fe.columns if any(k in c for k in ["lag", "roll", "growth"])]
assert not df_fe[lag_roll_cols].isnull().values.any(), "Fallo en PARTE 5: Aún hay NaNs en las columnas lag/roll."

assert 'País_Costa Rica' in df_fe.columns or 'País_costa rica' in df_fe.columns, "Fallo en PARTE 6: No se crearon las columnas dummy 'País_...'."
assert os.path.exists(fe_path), f"Fallo en PARTE 7: El archivo Parquet {fe_path} no se guardó."
assert 'País' in df_full_vista.columns, "Fallo en PARTE 8: La vista humana (df_full_vista) no se reconstruyó correctamente."
logging.info("✅ [Assert OK] Paso 2: Feature engineering, guardado y reconstrucción validados.")


## Paso 3: Entrenamiento de XGBoost y Forecasting Recursivo (2026-2030)

¡Este es el paso final! Aquí es donde el trabajo de los Pasos 1 y 2 da frutos.

### La Lógica de XGBoost (Diferente a Prophet)

A diferencia de Prophet, no necesitamos un loop para entrenar *cientos* de modelos. Gracias al Feature Engineering (Paso 2), entrenaremos **UN SOLO MODELO GLOBAL** que entiende todas las dinámicas (lags, crisis, y qué País/Sector es) a la vez.

### Plan de Acción (Paso 3)

1.  **Validación:** Entrenar un modelo con datos hasta 2023 y validarlo con 2024-2025 (nuestros datos más recientes) para probar que es preciso.
2.  **Modelo Final:** Re-entrenar el modelo con el 100% de nuestros datos (hasta 2025) para que tenga la máxima información posible.
3.  **Loop de Forecasting Recursivo:** Predecir 2026, usar esa predicción para calcular los features de 2027, predecir 2027, y así sucesivamente hasta 2030.
4.  **Guardar y Visualizar:** Formatear la predicción de 2026 con los colores y estilos que definimos.

---

**Siguiente celda → Ejecución del Entrenamiento y Forecasting 2026-2030.**

In [20]:
# =============================================
# --- PASO 3: ENTRENAMIENTO Y FORECASTING XGBOOST ---
# =============================================
import warnings
warnings.filterwarnings("ignore")

# =============================================
# PASO 3 - PARTE 1/8: Importar librerías
# =============================================
logging.info("\n--- PASO 3 - PARTE 1/8: Importar librerías ---")
# Ya importados en la Celda 1, pero confirmamos
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
import json
logging.info("Librerías listas: XGBoost + Sklearn (Metrics)")

In [21]:
# =============================================
# PASO 3 - PARTE 2/8: Cargar datos de FEATURES
# =============================================
logging.info("\n--- PASO 3 - PARTE 2/8: Cargar datos de Features ---")
# Este es el archivo NUMÉRICO (One-Hot) creado en el Paso 2, Parte 7
fe_path = f"{OUTDIR}/tabla_final_features.parquet"
# df_full_vista (con texto) también se cargará, pero para el loop
vista_path = f"{OUTDIR}/vista_todos_paises_features.parquet"

try:
    df_model = pd.read_parquet(fe_path)
    df_vista = pd.read_parquet(vista_path) # Para los nombres de grupos
except FileNotFoundError:
    logging.error(f"ERROR: No se encontró {fe_path}. Ejecuta el Paso 2 primero.")
    # raise

logging.info(f"Datos de features (para ML) cargados: {df_model.shape}")
logging.info(f"Datos de Vista (para loop) cargados: {df_vista.shape}")

In [22]:
# =============================================
# PASO 3 - PARTE 3/8: Definir Features (X) y Target (y)
# =============================================
logging.info("\n--- PASO 3 - PARTE 3/8: Definir Features (X) y Target (y) ---")
TARGET = "Monto Total (USD)"

# Features son TODAS las columnas MENOS el target y las de texto originales (que ya quitamos)
# 'Año' SÍ es una feature importante
FEATURES = [col for col in df_model.columns if col != TARGET]

logging.info(f"Target: {TARGET}")
logging.info(f"Total de Features: {len(FEATURES)}")
logging.info(f"Ejemplo de Features: {FEATURES[:3]}... {FEATURES[-3:]}")

In [23]:
# =============================================
# PASO 3 - PARTE 4/8: Validación (Entrenar hasta 2023, Validar 2024-2025)
# =============================================
logging.info("\n--- PASO 3 - PARTE 4/8: Validación Rápida (Entrenar/Validar) ---")

# División temporal ESTRICTA
df_train = df_model[df_model['Año'] <= 2023]
df_val = df_model[df_model['Año'] > 2023]

X_train, y_train = df_train[FEATURES], df_train[TARGET]
X_val, y_val = df_val[FEATURES], df_val[TARGET]

logging.info(f"Filas Train (<= 2023): {len(X_train)}")
logging.info(f"Filas Val (2024-2025): {len(X_val)}")

model_val = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=5,
    early_stopping_rounds=50,
    random_state=42,
    n_jobs=-1
)

model_val.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_val, y_val)],
              verbose=False)

preds = model_val.predict(X_val)

mae = mean_absolute_error(y_val, preds)
rmse = np.sqrt(mean_squared_error(y_val, preds))
# Evitar división por cero en MAPE si y_val tiene ceros
y_val_no_cero = y_val[y_val != 0]
preds_no_cero = preds[y_val != 0]
mape = mean_absolute_percentage_error(y_val_no_cero, preds_no_cero) if len(y_val_no_cero) > 0 else 0.0
r2 = r2_score(y_val, preds)
y_val_mean = y_val.mean()

logging.info("\n--- Métricas de Validación (2024-2025) ---")
logging.info(f"Media del Monto (2024-2025): ${y_val_mean:,.0f}")
logging.info(f"MAE  (Error Absoluto Medio):   ${mae:,.0f} (Error promedio por aprobación)")
logging.info(f"RMSE (Raíz Error Cuadrático): ${rmse:,.0f} (Penaliza errores grandes)")
logging.info(f"MAPE (Error Porcentual Medio): {mape:.2%}")
logging.info(f"R²   (Coef. de Determinación): {r2:.2f}")
logging.info("\n¡Modelo validado! Ahora entrenamos el modelo final con TODOS los datos.")

In [24]:
# =============================================
# PASO 3 - PARTE 5/8: Entrenar Modelo FINAL (con 100% de datos)
# =============================================
logging.info("\n--- PASO 3 - PARTE 5/8: Entrenar Modelo FINAL (Datos hasta 2025) ---")

X_full, y_full = df_model[FEATURES], df_model[TARGET]

# Usamos los 'mejores' hiperparámetros de la validación
best_iterations = model_val.best_iteration if model_val.best_iteration > 0 else 500

model_final = xgb.XGBRegressor(
    n_estimators=best_iterations, # Entrenar con el número óptimo de árboles
    learning_rate=0.01,
    max_depth=5,
    random_state=42,
    n_jobs=-1
)

model_final.fit(X_full, y_full, verbose=False)
logging.info(f"¡Modelo final entrenado con {best_iterations} estimadores!")

In [25]:
logging.info("Guardando hiperparámetros del modelo final...")
model_params = {
    'n_estimators': best_iterations,
    'learning_rate': 0.01,
    'max_depth': 5,
    'random_state': 42,
    'objective': 'reg:squarederror' # Objetivo de regresión
}
params_path = f"{OUTDIR}/model_final_hyperparameters.json"
with open(params_path, 'w') as f:
    json.dump(model_params, f, indent=4)
logging.info(f"Hiperparámetros guardados en: {params_path}")

In [26]:
# =============================================
# PASO 3 - PARTE 6/8: Loop de Forecasting Recursivo (2026-2030)
# =============================================
logging.info("\n--- PASO 3 - PARTE 6/8: Loop de Forecasting Recursivo (2026-2030) ---")

# Necesitamos df_ml (con texto) para obtener los grupos y el último año
df_ml_asc = df_ml.sort_values('Año')
group_cols = ["País", "Sector Institucional", "Tipo de Socio"]

# 1. Obtener la "plantilla" de grupos únicos
# Usamos el último año (2025) como base
grupos_futuros = df_ml[df_ml['Año'] == df_ml['Año'].max()][group_cols].drop_duplicates()
logging.info(f"Generando esqueleto para {len(grupos_futuros)} grupos únicos...")

# 2. DataFrame para guardar predicciones
df_forecast = df_vista.copy() # df_vista tiene texto Y features (de Paso 2)
predicciones = []

YEARS_TO_FORECAST = [2026, 2027, 2028, 2029, 2030]

for year in YEARS_TO_FORECAST:
    logging.info(f"--- Prediciendo {year} ---")

    # 1. Crear el "esqueleto" para el nuevo año
    df_new_year = grupos_futuros.copy()
    df_new_year["Año"] = year

    # 2. Crear Features (Lags, Rolling, Crisis) para este *nuevo* año
    # Necesitamos el DataFrame completo (historia + predicciones) para calcular esto
    df_temp_history = df_forecast.sort_values('Año')

    # --- Lags (basados en el año anterior) ---
    for lag in [1, 2, 3]:
        # Unir datos del año (año - lag)
        df_lag = df_temp_history[df_temp_history['Año'] == year - lag][group_cols + [TARGET, 'Cantidad Total', 'Promedio por Aprobación (USD)']]
        df_lag = df_lag.rename(columns={
            TARGET: f"monto_lag_{lag}",
            'Cantidad Total': f"cant_lag_{lag}",
            'Promedio por Aprobación (USD)': f"prom_lag_{lag}"
        })
        df_new_year = pd.merge(df_new_year, df_lag, on=group_cols, how='left')

    # --- Rolling (basados en los 3 años anteriores) ---
    df_roll = df_temp_history[df_temp_history['Año'].between(year - 3, year - 1)]
    df_roll_agg = df_roll.groupby(group_cols).agg(
        monto_roll_mean_3=('Monto Total (USD)', 'mean'),
        monto_roll_std_3=('Monto Total (USD)', 'std'),
        cant_roll_mean_3=('Cantidad Total', 'mean')
    ).reset_index()
    df_new_year = pd.merge(df_new_year, df_roll_agg, on=group_cols, how='left')

    # --- Growth (basados en año anterior y 3 años antes) ---
    df_lag_1 = df_temp_history[df_temp_history['Año'] == year - 1][group_cols + [TARGET]]
    df_lag_3 = df_temp_history[df_temp_history['Año'] == year - 3][group_cols + [TARGET]]
    df_new_year = pd.merge(df_new_year, df_lag_1.rename(columns={TARGET: 'monto_prev_1y'}), on=group_cols, how='left')
    df_new_year = pd.merge(df_new_year, df_lag_3.rename(columns={TARGET: 'monto_prev_3y'}), on=group_cols, how='left')

    # Rellenar NaNs de los cálculos
    df_new_year = df_new_year.fillna(0) # Rellenamos con 0 como en el training

    # Calcular growth (evitando división por cero)
    df_new_year["monto_growth_1y_grouped"] = np.where(df_new_year['monto_prev_1y'] != 0, (df_new_year['monto_prev_1y'] / df_new_year['monto_prev_1y']) - 1, 0) * 100
    df_new_year["monto_growth_3y_grouped"] = np.where(df_new_year['monto_prev_3y'] != 0, (df_new_year['monto_prev_3y'] / df_new_year['monto_prev_3y']) - 1, 0) * 100


    # --- Crisis Flags (para el año futuro) ---
    df_new_year["crisis_covid"]       = 0 # Ya pasó
    df_new_year["crisis_2008"]        = 0 # Ya pasó
    df_new_year["crisis_nicaragua"]   = ((df_new_year["Año"] >= 2018) & (df_new_year["País"] == "Nicaragua")).astype(int) # Podría seguir
    df_new_year["post_covid_boom"]    = 1 # Seguimos en post-boom

    # 3. One-Hot Encoding (igual que en training)
    df_new_year_dummies = pd.get_dummies(df_new_year[cat_cols], drop_first=True, dtype=int)
    df_new_year_features = pd.concat([df_new_year.drop(columns=cat_cols), df_new_year_dummies], axis=1)

    # 4. Alinear columnas (¡MUY IMPORTANTE!)
    # Asegurar que las columnas (features) sean exactamente las mismas que en X_full
    X_pred = pd.DataFrame(columns=X_full.columns, index=df_new_year_features.index)

    # Copiar las columnas que coinciden
    common_cols_model = list(set(X_full.columns) & set(df_new_year_features.columns))
    X_pred[common_cols_model] = df_new_year_features[common_cols_model]

    # Asegurar que las columnas (features) sean exactamente las mismas que en X_full
    if not X_pred.columns.equals(X_full.columns):
        logging.warning(f"ADVERTENCIA: Desajuste de columnas en el año {year}. Re-alineando...", file=sys.stderr)
        # Re-alinea X_pred al orden exacto de X_full, rellenando con 0 las que falten
        X_pred = X_pred.reindex(columns=X_full.columns, fill_value=0)

    # Rellenar con 0 cualquier NaN restante
    X_pred = X_pred.fillna(0)

    # 5. ¡PREDECIR!
    prediccion_monto = model_final.predict(X_pred[FEATURES])

    # 6. Guardar la predicción
    df_new_year[TARGET] = prediccion_monto
    # Asumimos que Cantidad y Promedio son 0 o NaN (solo predecimos Monto)
    df_new_year['Cantidad Total'] = 0
    df_new_year['Promedio por Aprobación (USD)'] = 0

    # Limpiamos predicciones negativas y alertamos si ocurre
    predicciones_negativas = (prediccion_monto < 0).sum()
    if predicciones_negativas > 0:
        logging.warning(f"ADVERTENCIA (Año {year}): Se generaron {predicciones_negativas} predicciones negativas, fueron ajustadas a 0.")
    df_new_year[TARGET] = np.maximum(0, prediccion_monto)

    # 7. Añadir la predicción al historial (para el próximo loop)
    # Seleccionamos solo las columnas que coinciden con df_forecast
    cols_to_append = [col for col in df_forecast.columns if col in df_new_year.columns]
    df_forecast = pd.concat([df_forecast, df_new_year[cols_to_append]], ignore_index=True)

    predicciones.append(df_new_year)

logging.info(f"\n¡{len(predicciones) * len(grupos_futuros)} PREDICCIONES CREADAS! (2026-2030 listas)")

# Consolidar todas las predicciones
preds_df = pd.concat(predicciones, ignore_index=True)

# Renombrar para la salida
preds_df = preds_df.rename(columns={
    TARGET: "Predicción",
    "País": "País",
    "Sector Institucional": "Sector",
    "Tipo de Socio": "Tipo"
})


In [27]:
# =============================================
# PASO 3 - PARTE 7/8: Vista bonita 2026
# =============================================
logging.info("\n--- PASO 3 - PARTE 7/8: Vista 2026 (ordenada y bonita) ---")

vista_2026 = preds_df[preds_df["Año"] == 2026][["País", "Sector", "Tipo", "Predicción"]].copy()
# Añadir placeholders para 'Inferior' y 'Superior' ya que XGBoost no da CI (Confidence Interval)
vista_2026 = vista_2026.sort_values("Predicción", ascending=False)


styled_2026 = vista_2026.style.format({
    "Predicción": lambda x: f"{x:,.0f}"
}).set_properties(**{
    'text-align': 'center',
    'color': 'black',
    'background-color': 'white',
    'font-size': '12pt'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#1f77b4'), ('color', 'white'), ('font-weight', 'bold')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f2f2f2')]}
]).background_gradient(cmap='Greens', subset=["Predicción"])

logging.info("PREDICCIONES 2026 (ordenadas de mayor a menor)")
display(styled_2026)

,País,Sector,Tipo,Predicción
10,Costa Rica,Sector Público,Fundadores,"99,956,584"
5,Honduras,Sector Público,Fundadores,"93,294,064"
7,El Salvador,Sector Público,Fundadores,"72,797,720"
11,Nicaragua,Sector Público,Fundadores,"56,759,936"
2,República Dominicana,Sector Público,Regional No Fundadores,"10,472,980"
9,Colombia,Sector Público,Extraregionales,"2,551,493"
4,Guatemala,Sector Público,Fundadores,"1,983,189"
0,Panamá,Sector Público,Regional No Fundadores,"1,637,170"
6,Regional,Sector Público,Regionales,"1,541,762"
1,Panamá,Sector Privado,Regional No Fundadores,"1,269,843"


In [28]:
# =============================================
# PASO 3 - PARTE 8/8: Guardar
# =============================================
logging.info("\n--- PASO 3 - PARTE 8/8: Guardar archivos ---")

# Seleccionar columnas limpias para guardar
cols_finales = ["Año", "País", "Sector", "Tipo", "Predicción"]
preds_df_final = preds_df[cols_finales].copy()

csv_pred = f"{OUTDIR}/predicciones_XGBOOST_2026_2030.csv"
excel_pred = f"{OUTDIR}/predicciones_XGBOOST_2026_2030.xlsx"
parquet_pred = f"{OUTDIR}/predicciones_XGBOOST_2026_2030.parquet"

preds_df_final.to_csv(csv_pred, index=False, encoding="utf-8")
preds_df_final.to_excel(excel_pred, index=False, engine="openpyxl")
preds_df_final.to_parquet(parquet_pred, index=False)

logging.info("¡GUARDADO EN 3 FORMATOS!")
logging.info(f"CSV     → {csv_pred}")
logging.info(f"Excel   → {excel_pred}")
logging.info(f"Parquet → {parquet_pred}")

logging.info("\n--- ¡CASO 2 (XGBoost) FINALIZADO! ---")
logging.info("Tienes predicciones 2026-2030 para TODOS los países")
logging.info("Archivos listos para Power BI / Tableau")

logging.info("\nEscribe: 'Listo, siguiente caso'")

In [29]:
logging.info("\n--- [TEST] Validando Paso 3 ---")
assert 'model_final' in locals(), "Paso 3.5 falló: El 'model_final' no fue entrenado."
assert 'preds_df' in locals() and not preds_df.empty, "Paso 3.6 falló: No se generaron predicciones (preds_df está vacío)."
assert 'Año' in preds_df.columns and preds_df['Año'].min() == 2026, "Paso 3.6 falló: Las predicciones no comienzan en 2026."
assert os.path.exists(parquet_pred), f"Paso 3.8 falló: El archivo de predicciones {parquet_pred} no se guardó."
logging.info("✅ [Assert OK] Paso 3: Entrenamiento, loop recursivo y guardado de predicciones validados.")

## Paso 4 (Final): DataFrame Unificado (Histórico + Pronóstico XGBoost)

¡Este es el paso final! Vamos a combinar nuestros datos históricos (1961-2025) con las predicciones de XGBoost (2026-2030) en un solo DataFrame maestro, listo para Power BI.

### ¿Qué generará esta celda?
Se generará un **DataFrame único y dinámico** (`df_unico`) que combina:

-   **Datos históricos reales** (1961–2025).
-   **Predicciones XGBoost** para 2026–2030.
-   **Continuidad perfecta** en gráficos: Usaremos el truco de duplicar el último año real (2025) como el primer punto del pronóstico (2026-01-01) para que las líneas en Power BI no se rompan.
-   **100% automático**: Funciona sin importar el año.

### Columnas del DataFrame final (XGBoost)
| Columna | Descripción |
|---|---|
| `Fecha` | Fecha (31-dic para históricos, 31-dic para pronósticos) |
| `Año` | Año numérico |
| `Sector Institucional` | Público / Privado |
| `País` | País beneficiario |
| `Tipo de Socio` | Fundadores, Regional No Fundadores, etc. |
| `Monto Total (USD)` | Valor real histórico (NaN en pronósticos) |
| `Datos` | "Reales" o "Predicción" |
| `Predicción` | Valor pronosticado (igual al real en históricos) |

---

**Siguiente celda → Ejecución de la unificación y guardado final.**

In [30]:
# =============================================
# --- PASO 4: UNIFICACIÓN (HISTÓRICO + XGBOOST) ---
# =============================================
logging.info("\n--- PASO 4: Iniciando unificación de Histórico + Predicciones XGBoost ---")

# =============================================
# 1. Último año histórico automático
# =============================================
logging.info("Paso 1/9: Detectando último año real...")
# df_ml fue cargado en el Paso 1 de este notebook
ultimo_año_real = df_ml["Año"].max()
logging.info(f"Último año con datos reales: {ultimo_año_real}")

# =============================================
# 2. Histórico preparado
# =============================================
logging.info("Paso 2/9: Preparando DataFrame histórico...")
hist_df = df_ml.copy()
hist_df["Datos"] = "Reales"
hist_df["Predicción"] = hist_df["Monto Total (USD)"] # La predicción de un dato real es él mismo
hist_df["Fecha"] = pd.to_datetime(hist_df["Año"].astype(str) + "-12-31")

# =============================================
# 3. Predicciones preparadas (¡AJUSTE XGBOOST!)
# =============================================
logging.info("Paso 3/9: Preparando DataFrame de predicciones (XGBoost)...")

# preds_df (de la celda anterior, Paso 3) ya tiene: Año, País, Sector, Tipo, Predicción
preds_df["Fecha"] = pd.to_datetime(preds_df["Año"].astype(str) + "-12-31") # Consistente
preds_df["Datos"] = "Predicción"
preds_df["Monto Total (USD)"] = pd.NA # El monto real es Nulo

# Renombrar columnas de grupo para que coincidan con hist_df
preds_df = preds_df.rename(columns={
    "Sector": "Sector Institucional",
    "Tipo": "Tipo de Socio"
})

logging.info("Predicciones XGBoost formateadas.")
if "País" not in preds_df.columns:
    logging.error("ERROR: ¡La columna 'País' falta en las predicciones!")
else:
    logging.info("¡Verificado! 'País' está en las predicciones.")


# =============================================
# 4. Columnas comunes
# =============================================
logging.info("Paso 4/9: Estandarizando columnas...")
# Definimos las columnas FINALES (sin Inferior/Superior)
common_cols = ["Fecha", "Año", "Sector Institucional", "País", "Tipo de Socio",
               "Monto Total (USD)", "Datos", "Predicción"]

hist_df = hist_df.reindex(columns=common_cols, fill_value=pd.NA)
preds_df = preds_df.reindex(columns=common_cols, fill_value=pd.NA)

# =============================================
# 5. Unir
# =============================================
logging.info("Paso 5/9: Concatenando Histórico + Predicciones...")
df_unico = pd.concat([hist_df, preds_df], ignore_index=True)

# =============================================
# 6. Duplicar último real (El truco para unir gráficos)
# =============================================
logging.info("Paso 6/9: Creando 'puente' visual (2025 -> 2026)...")
# Tomamos el dato real de 2025
ultimo_real = df_unico[(df_unico["Datos"] == "Reales") & (df_unico["Año"] == ultimo_año_real)].copy()
# Lo etiquetamos como "Predicción"
ultimo_real["Datos"] = "Predicción"
# Le asignamos la fecha del 31-Dic-2025 para que se "solape"
ultimo_real["Fecha"] = pd.to_datetime(str(ultimo_año_real) + "-12-31")

# El resto de valores (Predicción) ya son correctos (son = Monto Real)
df_unico = pd.concat([df_unico, ultimo_real], ignore_index=True)

# =============================================
# 7. Ordenar
# =============================================
logging.info("Paso 7/9: Ordenando el DataFrame final...")
df_unico = df_unico.sort_values(["País", "Sector Institucional", "Tipo de Socio", "Fecha"]).reset_index(drop=True)

# =============================================
# 8. Guardar
# =============================================
print("Paso 8/9: Guardando archivos finales unificados...")

print("Filtrando el 'puente' duplicado de 2025 para la exportación...")
df_para_exportar = df_unico[
    ~((df_unico['Datos'] == 'Predicción') & (df_unico['Año'] == ultimo_año_real))
].copy()
print(f"Filas en dashboard (con puente): {len(df_unico)}. Filas a exportar (sin puente): {len(df_para_exportar)}")

csv_unico = f"{OUTDIR}/bcies_aprobaciones_XGBOOST_historico_pronostico.csv"
excel_unico = f"{OUTDIR}/bcies_aprobaciones_XGBOOST_historico_pronostico.xlsx"
parquet_unico = f"{OUTDIR}/bcies_aprobaciones_XGBOOST_historico_pronostico.parquet"

# Usamos el nuevo DataFrame filtrado para guardar
df_para_exportar.to_csv(csv_unico, index=False, encoding="utf-8")
df_para_exportar.to_excel(excel_unico, index=False, engine="openpyxl")
df_para_exportar.to_parquet(parquet_unico, index=False)

logging.info(f"¡GUARDADO UNIFICADO (para Power BI)!")
logging.info(f"CSV     → {csv_unico}")
logging.info(f"Excel   → {excel_unico}")
logging.info(f"Parquet → {parquet_unico}")

# =============================================
# 9. Vista Costa Rica (SIN Inferior/Superior)
# =============================================
logging.info("\n--- PASO 9/9: Vista de Ejemplo (Costa Rica, 2020-2026) ---")

# Filtramos para ver la transición
cr = df_unico[(df_unico["País"] == "Costa Rica") & (df_unico['Año'] >= 2020)].copy()
cr_vista = cr.tail(12)[["Fecha", "Año", "País", "Sector Institucional", "Tipo de Socio", "Monto Total (USD)", "Predicción", "Datos"]]

# Formateador simple
def fmt_na(x, fmat="{:,.0f}"):
    if pd.isna(x):
        return "" # Vacío en lugar de N/A
    try:
        return fmat.format(x)
    except (ValueError, TypeError):
        return str(x)

styled_cr = cr_vista.style.format({
    "Monto Total (USD)": lambda x: fmt_na(x),
    "Predicción": lambda x: fmt_na(x),
}).set_properties(**{'text-align': 'center'})

display(styled_cr)

logging.info("\n¡Listo para visualización ejecutiva! Carga el archivo en Power BI y crea el dashboard. 🚀")

Paso 8/9: Guardando archivos finales unificados...
Filtrando el 'puente' duplicado de 2025 para la exportación...
Filas en dashboard (con puente): 681. Filas a exportar (sin puente): 669


,Fecha,Año,País,Sector Institucional,Tipo de Socio,Monto Total (USD),Predicción,Datos
146,2020-12-31 00:00:00,2020,Costa Rica,Sector Público,Fundadores,"699,160,000","699,160,000",Reales
147,2021-12-31 00:00:00,2021,Costa Rica,Sector Público,Fundadores,"580,000,000","580,000,000",Reales
148,2022-12-31 00:00:00,2022,Costa Rica,Sector Público,Fundadores,"1,190,000,000","1,190,000,000",Reales
149,2023-12-31 00:00:00,2023,Costa Rica,Sector Público,Fundadores,"1,000,000,000","1,000,000,000",Reales
150,2024-12-31 00:00:00,2024,Costa Rica,Sector Público,Fundadores,"770,000,000","770,000,000",Reales
151,2025-12-31 00:00:00,2025,Costa Rica,Sector Público,Fundadores,"815,000,000","815,000,000",Reales
152,2025-12-31 00:00:00,2025,Costa Rica,Sector Público,Fundadores,"815,000,000","815,000,000",Predicción
153,2026-12-31 00:00:00,2026,Costa Rica,Sector Público,Fundadores,,"99,956,584",Predicción
154,2027-12-31 00:00:00,2027,Costa Rica,Sector Público,Fundadores,,"9,799,561",Predicción
155,2028-12-31 00:00:00,2028,Costa Rica,Sector Público,Fundadores,,"10,094,612",Predicción


In [31]:
logging.info("\n--- [TEST] Validando Paso 4 ---")
assert 'df_unico' in locals() and not df_unico.empty, "El DataFrame unificado df_unico está vacío."
assert 'Datos' in df_unico.columns, "La columna 'Datos' (Reales/Predicción) no se creó."
assert df_unico[df_unico['Datos'] == 'Reales']['Monto Total (USD)'].isnull().sum() == 0, "Hay NaNs en los montos históricos."
assert df_unico[df_unico['Datos'] == 'Predicción']['Predicción'].isnull().sum() == 0, "Hay NaNs en los montos de predicción."
assert os.path.exists(parquet_unico), f"Paso 4.8 falló: El archivo unificado {parquet_unico} no se guardó."
logging.info("✅ [Assert OK] Paso 4: Unificación, 'puente' y guardado validados.")

## Paso 5: Mapa de Calor de Predicciones (2026-2030)

¡Visualicemos el futuro! Vamos a crear una tabla resumen (Heatmap) que muestre el **Monto Total (USD)** predicho por **XGBoost** para cada combinación de `País` y `Tipo de Socio` para los años 2026 a 2030.

-   Los colores más oscuros (`YlGnBu`) indicarán montos de aprobación más altos.
-   Incluiremos **SUBTOTALES** automáticos por "Tipo de Socio".
-   Incluiremos un **TOTAL GENERAL** al final.
-   El formato será idéntico al que usamos en el "Paso 2" (headers oscuros, formato de miles) para consistencia.

---

**Siguiente celda → Ejecución de la Tabla Dinámica y Mapa de Calor.**

In [32]:
# =============================================
# --- PASO 5: MAPA DE CALOR PREDICCIONES 2026-2030 (XGBoost) ---
# =============================================
logging.info("\n--- PASO 5: Iniciando Mapa de Calor de Predicciones XGBoost ---")

# ==========================
# 1. Filtrar solo predicciones
# ==========================
# Usamos el df_unico del Paso 4, que ya tiene todo limpio.
# Nos aseguramos de excluir el "puente" (Año 2025 con datos 'Predicción')
df_pred = df_unico[
    (df_unico["Datos"] == "Predicción") &
    (df_unico["Año"] > ultimo_año_real) # 'ultimo_año_real' es de la celda anterior (ej. 2025)
].copy()

# ==========================
# 2. Tabla dinámica base
# ==========================
pivot = df_pred.pivot_table(
    index=["Tipo de Socio", "País"],
    columns="Año",
    values="Predicción",
    aggfunc="sum",
    fill_value=0
)

# Asegurar años como int y en orden
pivot.columns = pivot.columns.astype(int)
pivot = pivot[sorted(pivot.columns)]

# ==========================
# 3. SUBTOTALES Y TOTAL GENERAL
# ==========================
logging.info("Calculando Subtotales y Total General...")

# Subtotal por Tipo de Socio
subtotals = pivot.groupby(level=0).sum()
subtotals.index = pd.MultiIndex.from_tuples(
    [(tipo, "SUBTOTAL") for tipo in subtotals.index],
    names=pivot.index.names
)

# Total general
total_general = pd.DataFrame(pivot.sum()).T
total_general.index = pd.MultiIndex.from_tuples(
    [("TOTAL GENERAL", "")],
    names=pivot.index.names
)

# Concatenar todo
pivot_full = pd.concat([pivot, subtotals, total_general])

# ==========================
# 4. Ordenar filas
# ==========================
# Ordenar filas: países por Tipo de Socio + SUBTOTAL + TOTAL GENERAL
order = []
# Usamos df_pred (solo predicciones) para obtener los grupos únicos
for tipo in sorted(df_pred["Tipo de Socio"].unique()):
    countries = df_pred[df_pred["Tipo de Socio"] == tipo]["País"].unique()
    for pais in sorted(countries):
        order.append((tipo, pais))
    order.append((tipo, "SUBTOTAL"))
order.append(("TOTAL GENERAL", ""))

pivot_full = pivot_full.reindex(order)

# ==========================
# 5. FORMATO PLANO PARA ESTILO
# ==========================
pivot_plain = pivot_full.reset_index()
pivot_plain = pivot_plain.rename(
    columns={"Tipo de Socio": "Tipo de Socio", "País": "País"}
)

# ==========================
# 6. ESTILOS Y MAPA DE CALOR
# ==========================
logging.info("Aplicando estilos y mapa de calor...")

# Columnas numéricas para el mapa de calor
num_cols = pivot_plain.select_dtypes(include=[np.number]).columns

# Estilos base
styled = (
    pivot_plain.style
    .format(lambda x: f"${x:,.0f}" if isinstance(x, (int, float, np.integer, np.floating)) else x)
    .set_properties(**{
        'text-align': 'center',
        'font-weight': 'bold'
    })
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#2c3e50'), # Header oscuro
                                      ('color', 'white'),
                                      ('text-align', 'center')]},
        {'selector': 'td', 'props': [('border', '1px solid #ddd')]}
    ])
    .background_gradient(cmap='YlGnBu', subset=pd.IndexSlice[:, num_cols]) # Mapa de calor
)

# Filas SUBTOTAL: gris claro + texto negro
rows_subtotal = pivot_plain["País"] == "SUBTOTAL"
styled = styled.set_properties(
    subset=(rows_subtotal, slice(None)),
    **{'background-color': '#f0f0f0', 'color': 'black'}
)

# Fila TOTAL GENERAL: azul oscuro + texto blanco
rows_total = pivot_plain["Tipo de Socio"] == "TOTAL GENERAL"
styled = styled.set_properties(
    subset=(rows_total, slice(None)),
    **{'background-color': '#1f77b4', 'color': 'white'}
)

logging.info("\n" + "="*80)
logging.info(" MAPA DE CALOR: PREDICCIONES DE APROBACIONES (USD) 2026-2030")
logging.info("="*80)
display(styled)

try:
    png_path = f"{OUTDIR}/heatmap_predicciones_por_socio.png"
    logging.info(f"Exportando heatmap como PNG (300 dpi) a: {png_path}")
    dfi.export(styled, png_path, dpi=300)
except Exception as e:
    # Removed 'file=sys.stderr' as it's not a valid argument for logging.warning
    logging.warning(f"  ADVERTENCIA: No se pudo exportar el PNG. Error: {e}")

logging.info("\n¡CASO 1 (XGBoost) FINALIZADO!")

Año,Tipo de Socio,País,2026,2027,2028,2029,2030
0,Extraregionales,Argentina,"$1,221,372","$1,225,999","$113,016","$328,910","$328,910"
1,Extraregionales,Colombia,"$2,551,493","$512,334","$113,016","$328,910","$328,910"
2,Extraregionales,SUBTOTAL,"$3,772,865","$1,738,332","$226,032","$657,820","$657,820"
3,Fundadores,Costa Rica,"$99,956,584","$9,799,561","$10,094,612","$423,979","$328,910"
4,Fundadores,El Salvador,"$72,797,720","$1,924,616","$1,909,686","$406,763","$328,910"
5,Fundadores,Guatemala,"$2,497,679","$888,128","$735,673","$657,820","$657,820"
6,Fundadores,Honduras,"$93,294,064","$1,924,616","$1,909,686","$406,763","$328,910"
7,Fundadores,Nicaragua,"$56,759,936","$1,373,470","$2,134,126","$406,763","$328,910"
8,Fundadores,SUBTOTAL,"$325,305,983","$15,910,390","$16,783,782","$2,302,089","$1,973,461"
9,Regional No Fundadores,Panamá,"$2,907,014","$2,364,290","$441,926","$657,820","$657,820"


Please use the Async API instead.


In [33]:
# =============================================
# --- PASO 5 (Adicional): MAPA DE CALOR POR SECTOR INSTITUCIONAL ---
# =============================================
logging.info("\n--- PASO 5 (Adicional): Iniciando Mapa de Calor por SECTOR ---")

# df_pred (con predicciones > 2025) ya existe de la celda anterior

# ==========================
# 2. Tabla dinámica base (por SECTOR)
# ==========================
pivot_sector = df_pred.pivot_table(
    index=["Sector Institucional", "País"], # <-- CAMBIO AQUÍ
    columns="Año",
    values="Predicción",
    aggfunc="sum",
    fill_value=0
)

pivot_sector.columns = pivot_sector.columns.astype(int)
pivot_sector = pivot_sector[sorted(pivot_sector.columns)]

# ==========================
# 3. SUBTOTALES Y TOTAL GENERAL
# ==========================
logging.info("Calculando Subtotales y Total General (por Sector)...")

# Subtotal por Sector Institucional
subtotals_sector = pivot_sector.groupby(level=0).sum()
subtotals_sector.index = pd.MultiIndex.from_tuples(
    [(sector, "SUBTOTAL") for sector in subtotals_sector.index],
    names=pivot_sector.index.names
)

# Total general (es el mismo, pero lo recalculamos por claridad)
total_general_sector = pd.DataFrame(pivot_sector.sum()).T
total_general_sector.index = pd.MultiIndex.from_tuples(
    [("TOTAL GENERAL", "")],
    names=pivot_sector.index.names
)

pivot_full_sector = pd.concat([pivot_sector, subtotals_sector, total_general_sector])

# ==========================
# 4. Ordenar filas
# ==========================
order_sector = []
# Ordenar por Sector (Público/Privado)
for sector in sorted(df_pred["Sector Institucional"].unique()):
    countries = df_pred[df_pred["Sector Institucional"] == sector]["País"].unique()
    for pais in sorted(countries):
        order_sector.append((sector, pais))
    order_sector.append((sector, "SUBTOTAL"))
order_sector.append(("TOTAL GENERAL", ""))

pivot_full_sector = pivot_full_sector.reindex(order_sector)

# ==========================
# 5. FORMATO PLANO PARA ESTILO
# ==========================
pivot_plain_sector = pivot_full_sector.reset_index()
pivot_plain_sector = pivot_plain_sector.rename(
    columns={"Sector Institucional": "Sector Institucional", "País": "País"}
)

# ==========================
# 6. ESTILOS Y MAPA DE CALOR
# ==========================
logging.info("Aplicando estilos y mapa de calor (por Sector)...")

num_cols_sector = pivot_plain_sector.select_dtypes(include=[np.number]).columns

styled_sector = (
    pivot_plain_sector.style
    .format(lambda x: f"${x:,.0f}" if isinstance(x, (int, float, np.integer, np.floating)) else x)
    .set_properties(**{
        'text-align': 'center',
        'font-weight': 'bold'
    })
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#2c3e50'),
                                      ('color', 'white'),
                                      ('text-align', 'center')]},
        {'selector': 'td', 'props': [('border', '1px solid #ddd')]}
    ])
    # --- AJUSTE DE COLOR ---
    # Cambiado de 'RdYlGn' a 'Blues' para una vista más limpia y profesional
    .background_gradient(cmap='Blues', subset=pd.IndexSlice[:, num_cols_sector])
)

# Filas SUBTOTAL
rows_subtotal_sector = pivot_plain_sector["País"] == "SUBTOTAL"
styled_sector = styled_sector.set_properties(
    subset=(rows_subtotal_sector, slice(None)),
    **{'background-color': '#f0f0f0', 'color': 'black'}
)

# Fila TOTAL GENERAL
rows_total_sector = pivot_plain_sector["Sector Institucional"] == "TOTAL GENERAL"
styled_sector = styled_sector.set_properties(
    subset=(rows_total_sector, slice(None)),
    **{'background-color': '#1f77b4', 'color': 'white'}
)

logging.info("\n" + "="*80)
logging.info(" MAPA DE CALOR (POR SECTOR): PREDICCIONES DE APROBACIONES (USD) 2026-2030")
logging.info("="*80)
display(styled_sector)

try:
    png_path_sector = f"{OUTDIR}/heatmap_predicciones_por_sector.png"
    logging.info(f"Exportando heatmap (Sector) como PNG (300 dpi) a: {png_path_sector}")
    dfi.export(styled_sector, png_path_sector, dpi=300)
except Exception as e:
    # Removed 'file=sys.stderr' as it's not a valid argument for logging.warning
    logging.warning(f"  ADVERTENCIA: No se pudo exportar el PNG del sector. Error: {e}")

logging.info("\n¡Mejora Item #8 completada!")


Año,Sector Institucional,País,2026,2027,2028,2029,2030
0,Sector Privado,Guatemala,"$514,490","$375,794","$328,910","$328,910","$328,910"
1,Sector Privado,Panamá,"$1,269,843","$375,794","$328,910","$328,910","$328,910"
2,Sector Privado,SUBTOTAL,"$1,784,333","$751,589","$657,820","$657,820","$657,820"
3,Sector Público,Argentina,"$1,221,372","$1,225,999","$113,016","$328,910","$328,910"
4,Sector Público,Colombia,"$2,551,493","$512,334","$113,016","$328,910","$328,910"
5,Sector Público,Costa Rica,"$99,956,584","$9,799,561","$10,094,612","$423,979","$328,910"
6,Sector Público,El Salvador,"$72,797,720","$1,924,616","$1,909,686","$406,763","$328,910"
7,Sector Público,Guatemala,"$1,983,189","$512,334","$406,763","$328,910","$328,910"
8,Sector Público,Honduras,"$93,294,064","$1,924,616","$1,909,686","$406,763","$328,910"
9,Sector Público,Nicaragua,"$56,759,936","$1,373,470","$2,134,126","$406,763","$328,910"


Please use the Async API instead.


In [34]:
logging.info("\n--- [TEST] Validando Paso 5 ---")
assert 'pivot_plain' in locals() and not pivot_plain.empty, "Paso 5 falló: El DataFrame 'pivot_plain' para el heatmap está vacío."
assert "TOTAL GENERAL" in pivot_plain["Tipo de Socio"].values, "Paso 5.3 falló: No se calculó el TOTAL GENERAL."
assert "SUBTOTAL" in pivot_plain["País"].values, "Paso 5.3 falló: No se calcularon los SUBTOTALES."
logging.info("✅ [Assert OK] Paso 5: Generación de pivot table y subtotales validada.")

## Paso 6 (Dashboard): Análisis Interactivo (Histórico + Predicción XGBoost)

Llegamos al dashboard final. Usando `ipywidgets` y `plotly`, podemos explorar dinámicamente todos los datos históricos (1961-2025) y las predicciones de XGBoost (2026-2030) que acabamos de generar.

### ¿Qué hace este dashboard?
-   **Filtros Dinámicos:** Permite seleccionar cualquier combinación de `Tipo de Socio`, `Sector Institucional` y `País`.
-   **Gráfico Unificado:** Muestra la línea histórica (negra) y la predicción (roja punteada) en un solo gráfico. Gracias al "puente" que creamos en el Paso 4, las líneas se conectan perfectamente.
-   **KPIs Automáticos:** Calcula los montos del último año real, el último año pronosticado y la variación total.
-   **KPIs Anuales (YoY):** Muestra el monto predicho para cada año (2026-2030) y su variación contra el año anterior.

---

**Siguiente celda → Ejecución del Dashboard Interactivo.**

In [35]:
# ==========================================================
# PASO 6: INTERACTIVO BCIE (XGBOOST)
# ==========================================================

# 1. Instalar y habilitar
logging.info("Instalando/Actualizando plotly e ipywidgets...")
!pip install -q plotly ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

# 2. Importaciones
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import logging
import sys
from functools import cache  # Mejora (Ítem #11) para caching

# -----------------------------
# 1. Constantes y Opciones Base
# -----------------------------
# Define las etiquetas para las opciones de "seleccionar todo"
ALL_TIPO = "Todos los tipos de socio"
ALL_SECTOR = "Todos los sectores"
ALL_PAIS = "Todos los países"

# Carga las opciones de filtro desde el DataFrame unificado (df_unico del Paso 4)
tipos_unicos = sorted(df_unico["Tipo de Socio"].dropna().unique())
sectores_unicos = sorted(df_unico["Sector Institucional"].dropna().unique())

# Prepara las listas finales para los widgets, añadiendo la opción "Todos"
tipos_opts = [ALL_TIPO] + tipos_unicos
sectores_opts = [ALL_SECTOR] + sectores_unicos

def obtener_paises(tipo_sel, sector_sel):
    """
    (Ítem #14) Filtra la lista de países disponibles basado
    en el Tipo de Socio y Sector.
    """
    df = df_unico.copy()

    # Aplica filtros si no son "Todos"
    if tipo_sel != ALL_TIPO:
        df = df[df["Tipo de Socio"] == tipo_sel]
    if sector_sel != ALL_SECTOR:
        df = df[df["Sector Institucional"] == sector_sel]

    paises = sorted(df["País"].dropna().unique())

    if not paises:
        return []

    # Devuelve la lista de países, añadiendo "Todos" si hay más de uno
    return [ALL_PAIS] + paises if len(paises) > 1 else paises

# Define los valores iniciales para los filtros
tipo_ini = ALL_TIPO
sector_ini = ALL_SECTOR
paises_ini = obtener_paises(tipo_ini, sector_ini)

# -----------------------------
# 2. Creación de Widgets (UI)
# -----------------------------

# Define un estilo estándar para los botones
button_width_px = '180px'
button_height_px = '45px'

# Widget para Tipo de Socio
widget_tipo = widgets.ToggleButtons(
    options=tipos_opts,
    description='',
    style={'button_width': button_width_px, 'font_weight': 'bold'},
    layout=widgets.Layout(
        width='100%',
        display='flex',
        flex_flow='row wrap',
        justify_content='flex-start',
        padding='4px 0px'
    )
)

# Widget para Sector Institucional
widget_sector = widgets.ToggleButtons(
    options=sectores_opts,
    description='',
    style={'button_width': button_width_px},
    layout=widgets.Layout(
        width='100%',
        display='flex',
        flex_flow='row wrap',
        justify_content='flex-start',
        padding='4px 0px'
    )
)

# Widget para País
widget_pais = widgets.ToggleButtons(
    options=paises_ini,
    description='',
    style={'button_width': button_width_px},
    layout=widgets.Layout(
        width='1400px',
        display='flex',
        flex_flow='row wrap',
        justify_content='flex-start',
        padding='4px 0px'
    )
)

# Widgets de salida: uno para el gráfico, otro para los KPIs en HTML
out = widgets.Output()
kpi_html = widgets.HTML()

# --- INICIO MEJORA (Plan item #10) ---
logging.info("Creando widget de rango de años...")
min_year = int(df_unico['Año'].min())
max_year = int(df_unico['Año'].max())

# Widget de Slider para Rango de Años
widget_años = widgets.IntRangeSlider(
    value=[max(min_year, max_year - 70), max_year], # Default: últimos 70 años
    min=min_year,
    max=max_year,
    step=1,
    description='Filtrar Años:',
    layout=widgets.Layout(width='500px')
)
# --- FIN MEJORA ---

# CSS para dar estilo a los botones (redondeados, color de selección)
custom_css = f"""
<style>
.widget-toggle-buttons .widget-toggle-button {{
    width: {button_width_px} !important;
    min-width: {button_width_px} !important;
    max-width: {button_width_px} !important;
    height: {button_height_px} !important;
    line-height: 1.2 !important;
    white-space: normal !important;
    margin: 8px 8px 8px 8px !important;
    border-radius: 8px !important;
}}
.widget-toggle-buttons .widget-toggle-button.mod-active,
.widget-toggle-buttons .widget-toggle-button.mod-selected {{
    background-color: #105682 !important;
    color: #ffffff !important;
    border-color: #105682 !important;
}}
</style>
"""
display(HTML(custom_css))

# -----------------------------
# 3. Helper: Filtrar y Agregar Datos
# -----------------------------
@cache  # --- MEJORA (Plan item #11): Cachea los resultados
def get_data(tipo_sel, sector_sel, pais_sel):
    """
    (Ítem #14) Filtra y agrupa el DataFrame 'df_unico' según la selección.
    Agrega los montos por Fecha, Año y Datos ('Reales'/'Predicción').
    Esta función está cacheada para mayor rendimiento.
    """
    df = df_unico.copy()

    # Aplica los 3 filtros principales
    if tipo_sel != ALL_TIPO:
        df = df[df["Tipo de Socio"] == tipo_sel]
    if sector_sel != ALL_SECTOR:
        df = df[df["Sector Institucional"] == sector_sel]
    if pais_sel != ALL_PAIS:
        df = df[df["País"] == pais_sel]
    if df.empty:
        return df

    # --- CORRECCIÓN (KeyError: 'Año') ---
    # Agrupamos por Año (además de Fecha y Datos) para preservarlo
    group_cols = ["Fecha", "Año", "Datos"]

    value_cols = ["Monto Total (USD)", "Predicción"]
    df_group = (
        df.groupby(group_cols, as_index=False)[value_cols]
          .sum(min_count=1) # min_count=1 preserva NaNs si todos son NaN
          .sort_values("Fecha")
    )
    return df_group

# -----------------------------
# 4. Lógica de KPIs Dinámicos
# -----------------------------
def actualizar_kpis(df, titulo_ctx):
    """(Ítem #14) Calcula y renderiza la barra superior de KPIs."""
    if df.empty:
        kpi_html.value = "<b>Sin datos para la selección actual.</b>"
        return

    # Separa datos reales y de predicción
    df_real = df[df["Datos"] == "Reales"].dropna(subset=["Monto Total (USD)"])
    df_pred = df[df["Datos"] == "Predicción"].dropna(subset=["Predicción"])

    # Encuentra el último valor REAL
    last_real_val, last_real_year = np.nan, None
    if not df_real.empty:
        last_real_row = df_real.sort_values("Fecha").iloc[-1]
        last_real_val = float(last_real_row["Monto Total (USD)"])
        last_real_year = int(last_real_row["Fecha"].year)

    # Encuentra el último valor PREDICHO
    last_pred_val, last_pred_year = np.nan, None
    if not df_pred.empty:
        df_pred_sorted = df_pred.sort_values("Fecha").copy()
        last_pred_row = df_pred_sorted.iloc[-1]
        last_pred_val = float(last_pred_row["Predicción"])
        last_pred_year = int(last_pred_row["Fecha"].year)

    # Calcula la variación total (Último Real vs. Última Predicción)
    growth_txt, growth_color = "-", "#bdc3c7"
    if (not np.isnan(last_real_val) and not np.isnan(last_pred_val) and last_real_val != 0):
        growth = (last_pred_val / last_real_val - 1) * 100
        growth_txt = f"{growth:,.1f}%"
        growth_color = "#27ae60" if growth >= 0 else "#c0392b"

    # Calcula las variaciones Año a Año (YoY) para el pronóstico
    yoy_html = ""
    if not df_pred.empty:
        real_map = {}
        if not df_real.empty:
            real_sorted = df_real.sort_values("Fecha").copy()
            real_sorted["Year"] = real_sorted["Fecha"].dt.year
            real_year = (real_sorted.groupby("Year", as_index=False).last())
            real_map = dict(zip(real_year["Year"], real_year["Monto Total (USD)"]))

        pred_sorted = df_pred.sort_values("Fecha").copy()
        pred_sorted["Year"] = pred_sorted["Fecha"].dt.year
        pred_year = (pred_sorted.groupby("Year", as_index=False).last().sort_values("Year"))
        pred_map = dict(zip(pred_year["Year"], pred_year["Predicción"]))
        years_pred = list(pred_year["Year"])

        # Excluye el año "puente" de la barra de KPIs YoY
        if last_real_year and len(years_pred) > 5:
            years_pred = [y for y in years_pred if y > last_real_year]

        # Construye los bloques HTML para cada año
        blocks = []
        for y in years_pred:
            curr_val = float(pred_map.get(y, np.nan))
            # El valor anterior puede ser una predicción (y-1) o el último real
            prev_val = pred_map.get(y - 1, real_map.get(y - 1))

            if prev_val is not None and not np.isnan(prev_val) and prev_val != 0:
                yoy = (curr_val / float(prev_val) - 1) * 100
                yoy_str = f"{yoy:,.1f}%"
                yoy_color = "#27ae60" if yoy >= 0 else "#c0392b"
            else:
                yoy_str = "-"
                yoy_color = "#bdc3c7"

            blocks.append(f"""
              <div style="margin-right:26px;">
                <div style="font-size:11px;color:#777;">{y}</div>
                <div style="font-size:13px;"><b>${curr_val:,.0f}</b></div>
                <div style="font-size:13px;color:{yoy_color};">{yoy_str}</div>
              </div>
            """)

        if blocks:
            yoy_html = (
                "<div style='display:flex;align-items:flex-end;"
                "margin-left:40px;border-left:1px solid #444;padding-left:24px;'>"
                + "".join(blocks) +
                "</div>"
            )

    # Compila el HTML final para la barra de KPIs
    html = f"""
    <div style="font-family:system-ui; display:flex; gap:40px; padding:8px 0; align-items:flex-end;">
      <div>
        <div style="font-size:11px;color:#777;">Contexto</div>
        <div style="font-size:14px;"><b>{titulo_ctx}</b></div>
      </div>
      <div>
        <div style="font-size:11px;color:#777;">Último año real</div>
        <div style="font-size:18px;"><b>{last_real_year if last_real_year else '-'}</b></div>
        <div style="font-size:14px;">
          {f'${last_real_val:,.0f}' if not np.isnan(last_real_val) else '-'}
        </div>
      </div>
      <div>
        <div style="font-size:11px;color:#777;">Último año pronosticado</div>
        <div style="font-size:18px;"><b>{last_pred_year if last_pred_year else '-'}</b></div>
        <div style="font-size:14px;">
          {f'${last_pred_val:,.0f}' if not np.isnan(last_pred_val) else '-'}
        </div>
      </div>
      <div>
        <div style="font-size:11px;color:#777;">Variación estimada</div>
        <div style="font-size:20px;color:{growth_color};">
          <b>{growth_txt}</b>
        </div>
      </div>
      {yoy_html}
    </div>
    """
    kpi_html.value = html

# -----------------------------
# 5. Construcción del Gráfico
# -----------------------------

FIG_WIDTH = 1400
FIG_HEIGHT = 450

def construir_figura(tipo_sel, sector_sel, pais_sel, year_range):
    """(Ítem #14) Genera la figura principal de Plotly (histórica + predicción)."""

    # 1. Obtener datos (cacheado)
    df = get_data(tipo_sel, sector_sel, pais_sel)

    # 2. Aplicar filtro de Rango de Años (Mejora Ítem #10)
    if not df.empty:
        df = df[
            (df['Año'] >= year_range[0]) &
            (df['Año'] <= year_range[1])
        ].copy()

    # 3. Crear título dinámico
    partes = [
        ("Todos los países" if pais_sel == ALL_PAIS else pais_sel),
        ("Todos los tipos de socio" if tipo_sel == ALL_TIPO else tipo_sel),
        ("Todos los sectores" if sector_sel == ALL_SECTOR else sector_sel)
    ]
    titulo_ctx = " · ".join(partes)

    fig = go.Figure()

    # 4. Manejar caso de "Sin Datos"
    if df.empty:
        fig.update_layout(
            title=f"Sin datos para {titulo_ctx}",
            template="simple_white",
            width=FIG_WIDTH,
            height=FIG_HEIGHT,
            autosize=False
        )
        actualizar_kpis(df, titulo_ctx)
        return fig, titulo_ctx

    # 5. Dibujar Línea Histórica (Reales)
    df_real = df[df["Datos"] == "Reales"]
    if not df_real.empty and df_real["Monto Total (USD)"].notna().any():
        fig.add_trace(go.Scatter(
            x=df_real["Fecha"],
            y=df_real["Monto Total (USD)"],
            mode="lines+markers",
            name="Monto Aprobado",
            line=dict(color="black"),
            marker=dict(symbol="diamond"),
            line_shape="spline"
        ))

    # 6. Dibujar Línea de Pronóstico (XGBoost)
    df_pred = df[df["Datos"] == "Predicción"]
    if not df_pred.empty and df_pred["Predicción"].notna().any():
        df_pred = df_pred.sort_values("Fecha")

        # Dibuja la línea roja punteada
        fig.add_trace(go.Scatter(
            x=df_pred["Fecha"],
            y=df_pred["Predicción"],
            mode="lines",
            name="Predicción (XGBoost)",
            line=dict(color="red", dash="dot"),
            line_shape="spline"
        ))

        # Dibuja la región sombreada y la línea vertical
        inicio_pred = df_pred["Fecha"].min()
        fin_pred = df_pred["Fecha"].max()
        fig.add_vrect(
            x0=inicio_pred, x1=fin_pred,
            fillcolor="rgba(231,76,60,0.03)",
            line_width=0, layer="below"
        )
        fig.add_vline(
            x=inicio_pred,
            line_width=1,
            line_dash="dot",
            line_color="gray"
        )

    # 7. Configuración final del Layout
    fig.update_layout(
        title=f"Aprobaciones y Predicciones BCIE (XGBoost) - {titulo_ctx}",
        xaxis_title="Año",
        yaxis_title="Monto Total (USD)",
        template="simple_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
        margin=dict(l=60, r=30, t=70, b=50),
        width=FIG_WIDTH,
        height=FIG_HEIGHT,
        autosize=False
    )

    # 8. Actualizar KPIs
    actualizar_kpis(df, titulo_ctx)
    return fig, titulo_ctx

# -----------------------------
# 6. Callbacks (El "Cableado")
# -----------------------------

def on_filtros_change(change):
    """Actualiza la lista de países cuando cambia el Tipo o Sector."""
    tipo_sel = widget_tipo.value
    sector_sel = widget_sector.value
    nuevos_paises = obtener_paises(tipo_sel, sector_sel)

    if not nuevos_paises:
        widget_pais.options = []
        widget_pais.value = None
    else:
        widget_pais.options = nuevos_paises
        if ALL_PAIS in nuevos_paises:
            widget_pais.value = ALL_PAIS
        else:
            widget_pais.value = nuevos_paises[0]
    actualizar_grafico() # Llama al gráfico (que usará el nuevo país)

@out.capture(clear_output=True) # Captura la salida para mostrarla en el widget 'out'
def actualizar_grafico(change=None):
    """Función principal que redibuja el gráfico y los KPIs."""
    tipo_sel = widget_tipo.value
    sector_sel = widget_sector.value
    pais_sel = widget_pais.value
    year_range = widget_años.value # Mejora (Ítem #10)

    if not pais_sel:
        # Usa 'print' aquí porque 'logging' no es capturado por @out.capture
        print("Sin países disponibles para esta combinación.")
        kpi_html.value = ""
        return

    # Llama a la función de construcción
    fig, _ = construir_figura(tipo_sel, sector_sel, pais_sel, year_range)
    fig.show()

# Conecta los widgets a las funciones
widget_tipo.observe(on_filtros_change, names="value")
widget_sector.observe(on_filtros_change, names="value")
widget_pais.observe(actualizar_grafico, names="value")
widget_años.observe(actualizar_grafico, names="value") # Mejora (Ítem #10)

# -----------------------------
# 7. Renderizado Final
# -----------------------------
# Muestra todos los widgets en orden

display(HTML("<h3>Tipo de Socio</h3>"))
display(widget_tipo)

display(HTML("<h3>Sector Institucional</h3>"))
display(widget_sector)

display(HTML("<h3>País</h3>"))
display(widget_pais)

display(HTML("<h3>Rango de Años</h3>"))
display(widget_años) # Mejora (Ítem #10)

display(kpi_html) # Muestra la barra de KPIs

with out:
    actualizar_grafico() # Carga el gráfico inicial
display(out) # Muestra el widget del gráfico

ToggleButtons(layout=Layout(display='flex', flex_flow='row wrap', justify_content='flex-start', padding='4px 0…

ToggleButtons(layout=Layout(display='flex', flex_flow='row wrap', justify_content='flex-start', padding='4px 0…

ToggleButtons(layout=Layout(display='flex', flex_flow='row wrap', justify_content='flex-start', padding='4px 0…

IntRangeSlider(value=(1961, 2030), description='Filtrar Años:', layout=Layout(width='500px'), max=2030, min=19…

HTML(value='')

Output()

In [36]:
logging.info("\n--- [TEST] Validando Paso 6 ---")
assert 'widget_tipo' in locals(), "Paso 6 falló: El widget 'widget_tipo' no fue creado."
assert 'kpi_html' in locals(), "Paso 6 falló: El widget 'kpi_html' no fue creado."
assert 'df_unico' in locals() and not df_unico.empty, "Paso 6 falló: El DataFrame 'df_unico' no está disponible para el dashboard."
logging.info("✅ [Assert OK] Paso 6: Creación de widgets del dashboard validada.")